In [105]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import pickle

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

## Step 1: Read in hold out data, scalers, and best model

In [106]:
# holdout = pd.read_csv('resources/movies_holdout_features.csv', index_col=0)

In [107]:
# final_scaler = read_pickle(filename)
# final_model = read_pickle(filename)

In [108]:
holdout = pd.read_csv('kc_house_data_test_features.csv', index_col=0)
holdout

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,263000018,20140521T000000,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
4319,6600060120,20150223T000000,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
4320,1523300141,20140623T000000,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
4321,291310100,20150116T000000,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [109]:
pickle_in = open("model.pickle", "rb")
model = pickle.load(pickle_in)

In [110]:
len(model.coef_)

350

In [111]:
type(model)

sklearn.linear_model._base.LinearRegression

In [112]:
type(feature)

sklearn.preprocessing._data.PolynomialFeatures

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [113]:
holdout

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,263000018,20140521T000000,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
4319,6600060120,20150223T000000,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
4320,1523300141,20140623T000000,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
4321,291310100,20150116T000000,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [114]:
holdout['bedrooms'] = holdout['bedrooms'].replace([0,33,11],[holdout['bedrooms'].median(),3, 4])
holdout['bedrooms']

0       4.0
1       4.0
2       3.0
3       3.0
4       4.0
       ... 
4318    3.0
4319    4.0
4320    2.0
4321    3.0
4322    2.0
Name: bedrooms, Length: 4323, dtype: float64

In [115]:
holdout['age'] = 2020 - holdout['yr_built']

In [116]:
holdout['renovation_age'] = np.where(holdout['yr_renovated']>0, 2020 - holdout['yr_renovated'], 0)

In [117]:
df_dummy = pd.get_dummies(holdout, columns=['condition', 'view', 'grade'], drop_first=True)

In [118]:
features = ['sqft_living', 'sqft_lot', 'view_3', 'view_1', 'view_2', 'view_4', 'waterfront', 'condition_2', 'condition_3',
            'condition_4', 'condition_5', 'grade_4', 'grade_5','grade_6', 'grade_7', 'grade_8', 'grade_9',
           'grade_10', 'grade_11', 'grade_12', 'grade_13', 'yr_renovated', 'age', 'sqft_basement', 'sqft_above']

In [119]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_data = poly.fit_transform(df_dummy[features])
poly_data

array([[2.2700e+03, 1.1500e+04, 0.0000e+00, ..., 5.3290e+05, 1.1242e+06,
        2.3716e+06],
       [2.2700e+03, 1.1500e+04, 0.0000e+00, ..., 5.3290e+05, 1.1242e+06,
        2.3716e+06],
       [1.4700e+03, 1.7790e+03, 0.0000e+00, ..., 9.6100e+04, 3.5960e+05,
        1.3456e+06],
       ...,
       [1.0200e+03, 1.3500e+03, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0404e+06],
       [1.6000e+03, 2.3880e+03, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        2.5600e+06],
       [1.0200e+03, 1.0760e+03, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0404e+06]])

In [120]:
poly.get_feature_names(features)

['sqft_living',
 'sqft_lot',
 'view_3',
 'view_1',
 'view_2',
 'view_4',
 'waterfront',
 'condition_2',
 'condition_3',
 'condition_4',
 'condition_5',
 'grade_4',
 'grade_5',
 'grade_6',
 'grade_7',
 'grade_8',
 'grade_9',
 'grade_10',
 'grade_11',
 'grade_12',
 'grade_13',
 'yr_renovated',
 'age',
 'sqft_basement',
 'sqft_above',
 'sqft_living^2',
 'sqft_living sqft_lot',
 'sqft_living view_3',
 'sqft_living view_1',
 'sqft_living view_2',
 'sqft_living view_4',
 'sqft_living waterfront',
 'sqft_living condition_2',
 'sqft_living condition_3',
 'sqft_living condition_4',
 'sqft_living condition_5',
 'sqft_living grade_4',
 'sqft_living grade_5',
 'sqft_living grade_6',
 'sqft_living grade_7',
 'sqft_living grade_8',
 'sqft_living grade_9',
 'sqft_living grade_10',
 'sqft_living grade_11',
 'sqft_living grade_12',
 'sqft_living grade_13',
 'sqft_living yr_renovated',
 'sqft_living age',
 'sqft_living sqft_basement',
 'sqft_living sqft_above',
 'sqft_lot^2',
 'sqft_lot view_3',
 'sqft_

In [121]:
feature_in = open("feature.pickle", "rb")
feature = pickle.load(feature_in)
feature_in.close()

In [122]:
poly_data

array([[2.2700e+03, 1.1500e+04, 0.0000e+00, ..., 5.3290e+05, 1.1242e+06,
        2.3716e+06],
       [2.2700e+03, 1.1500e+04, 0.0000e+00, ..., 5.3290e+05, 1.1242e+06,
        2.3716e+06],
       [1.4700e+03, 1.7790e+03, 0.0000e+00, ..., 9.6100e+04, 3.5960e+05,
        1.3456e+06],
       ...,
       [1.0200e+03, 1.3500e+03, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0404e+06],
       [1.6000e+03, 2.3880e+03, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        2.5600e+06],
       [1.0200e+03, 1.0760e+03, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0404e+06]])

In [123]:
y_pred = model.predict(poly_data)
y_pred

array([528161.2206901 , 528161.2206901 , 438561.36170107, ...,
       307883.86777354, 417078.96636259, 305928.66843374])

In [82]:
# transformed_holdout = final_scaler(holdout)

## Step 3: Predict the holdout set

In [83]:
# final_answers = final_model.predict(transformed_holdout)

In [124]:
y_pred = model.predict(poly_data)
y_pred

array([528161.2206901 , 528161.2206901 , 438561.36170107, ...,
       307883.86777354, 417078.96636259, 305928.66843374])

## Step 4: Export your predictions

In [85]:
# final_answer.to_csv('housing_preds_your_name.csv')

In [125]:
pd.DataFrame(y_pred).to_csv('housing_pred_Raizel_Bernstein_1.csv')